In [8]:
# import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import io
from marmot.msg import ROS_SVO_Task
from enum import Enum
import copy


In [9]:
ENV = "house"

actions = np.load("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/0_"+ENV+"_actions.npy")
START = np.load("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/0_"+ENV+"_starts.npy")
GOALS = np.load("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/0_"+ENV+"_starts.npy")
neigh = np.load("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/0_"+ENV+"_neigh.npy")
svo_avg = np.load("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/0_"+ENV+"_svo_avg.npy")
svo_distri = np.load("/media/marmot-18/Thordak2TB/catkin_ws/src/Tanishq_MAPF/database/0_"+ENV+"_svo_distri.npy")

In [10]:
actions.shape, neigh.shape, svo_avg.shape, svo_distri.shape

((34, 10), (35, 10), (35, 10), (35, 10, 10))

In [11]:
TOPIC = None

class Status(Enum):
    STAGED = 200
    ENQUEUED = 201
    DONE = 202
    END = -3

class Task:
    # __actionDict__ = {0:np.array([0,0]), 1:np.array([0,1]), 2:np.array([1,0]), 3:np.array([0,-1]), 4:np.array([-1,0])}
    __actionDict__ = {0:np.array([0,0]), 1:np.array([1,0]), 2:np.array([0,1]), 3:np.array([-1,0]), 4:np.array([0,-1])}

    def __init__(self, tid, rid, start, action, time, neigh, svo_avg, svo_distri) -> None:
        self.taskID = tid
        self.robotID = rid
        self.action = action
        self.startPos = np.array(start)
        self.goalPos = np.array(self.__actionDict__[action]+start)
        self.time = time
        self.neigh = neigh-1
        self.svo_avg = svo_avg
        self.svo_distri = svo_distri
        self.preqs = dict()
        self.status = Status.STAGED
    
    def __publishTask__(self):
        global TOPIC
        message = ROS_SVO_Task()
        message.taskID = int(self.taskID)
        message.robotID = int(self.robotID)
        message.action = int(self.action)
        message.status = self.status.value
        message.neighbour = int(self.neigh)
        message.svo_avg = self.svo_avg
        message.svo_distri = self.svo_distri

        # print(message)
        # rospy.loginfo(message)
        TOPIC[self.robotID].publish(message)

    
    def __repr__(self) -> str:
        asd = "\n"
        for i in self.__dir__():
            if not i.startswith('__'):
                asd+=i
                asd+=": "
                asd+=str(getattr(self,i))
                asd+=", "

        return asd

class GraphTD:
    def __init__(self, taskList, startCells, neigh, svo_avg, svo_distri) -> None:
        positions = {}
        self.currentPositions = copy.deepcopy(startCells)

        self.graph = dict()
        self.taskDict = dict()
        tId = 1
        self.robotList = []
        for rid in range(len(startCells)):
        
            prevTask = None
            for i, task in enumerate(taskList[:,rid]):

                # print(tId, rid, currentPositions[rid], task, i)
                t = Task(tId, rid, self.currentPositions[rid], task, i,neigh[i,rid], svo_avg[i,rid], svo_distri[i,rid,:])
                # print(t)
                self.taskDict[tId] = t
                tId+=1
                self.currentPositions[rid] = t.goalPos
                self.addVertex(t)

                if(tuple(t.goalPos) not in positions):
                    positions[tuple(t.goalPos)] = dict()
                positions[tuple(t.goalPos)][t.time] = t.taskID

                if(prevTask is None):
                    self.robotList.append(t)
                else:
                    self.addEdge(prevTask, t)

                prevTask = t
        # print(self.taskDict)

        # self.pos = positions
        for pos in positions:
            timeList = sorted(positions[pos].keys())
            # print(timeList)
            # print(positions[pos][0])
            for i in range(len(timeList)):
                for j in range(i+1, len(timeList)):
                    # print(positions[pos], positions[pos][timeList[i]], positions[pos][timeList[j]])
                    self.addEdge(self.taskDict[positions[pos][timeList[i]]+1], self.taskDict[positions[pos][timeList[j]]])

        for t in range(1, len(self.taskDict), len(taskList)):
            # print(self.taskDict[t])
            if(tuple(self.taskDict[t].startPos) in positions):
                for time in positions[tuple(self.taskDict[t].startPos)]:
                    self.addEdge(self.taskDict[t], self.taskDict[positions[tuple(self.taskDict[t].startPos)][time]])



        for rid in range(len(startCells)):
            firstTid = self.robotList[rid].taskID
            for taskID in range(firstTid, firstTid+len(taskList)):
                if(taskID not in self.taskDict):
                    continue
                task = self.taskDict[taskID]
            
    
    def addVertex(self, task):
        assert type(task) == Task
        if(task.taskID not in self.graph):
            self.graph[task.taskID] = list()


    def addEdge(self, taskFrom, taskTo):
        assert  type(taskFrom)==Task and taskFrom.taskID in self.graph
        assert type(taskTo)==Task and taskTo.taskID in self.graph

        if(taskFrom.taskID in taskTo.preqs or taskFrom.taskID==taskTo.taskID):
            return
        for t in self.graph[taskFrom.taskID]:
            if(self.taskDict[t].robotID==taskTo.robotID):
                return


        taskTo.preqs[taskFrom.taskID] = taskFrom.status.value
        self.graph[taskFrom.taskID].append(taskTo.taskID)

    def changeStatus(self, taskID):
        # rospy.loginfo(taskID)
        task = self.taskDict[taskID]
        # print(taskID, task)
        task.status = Status(task.status.value+1)

        task.__publishTask__()

In [12]:
GRAPH = GraphTD(actions,START,neigh, svo_avg,svo_distri)

In [13]:
GRAPH.taskDict

{1: 
 taskID: 1, robotID: 0, action: 1.0, startPos: [3 4], goalPos: [4 4], time: 0, neigh: 4, svo_avg: 4.5, svo_distri: [0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1], preqs: {}, status: Status.STAGED, ,
 2: 
 taskID: 2, robotID: 0, action: 2.0, startPos: [4 4], goalPos: [4 5], time: 1, neigh: 4, svo_avg: 4.388974450528622, svo_distri: [0.11558668 0.07032368 0.11782993 0.10156279 0.12520382 0.10865778
  0.08655829 0.07698317 0.09967805 0.09761579], preqs: {1: 200}, status: Status.STAGED, ,
 3: 
 taskID: 3, robotID: 0, action: 2.0, startPos: [4 5], goalPos: [4 6], time: 2, neigh: 4, svo_avg: 3.853307366371155, svo_distri: [0.14333591 0.10708153 0.16338025 0.08705562 0.08980825 0.08359171
  0.10862256 0.07706392 0.07061791 0.06944232], preqs: {2: 200}, status: Status.STAGED, ,
 4: 
 taskID: 4, robotID: 0, action: 2.0, startPos: [4 6], goalPos: [4 7], time: 3, neigh: 4, svo_avg: 3.2831733748316765, svo_distri: [0.15590999 0.14463362 0.2044317  0.09756219 0.087145   0.07119866
  0.07952601 0.06